## instruction语料微调llama3-8b

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

### 之前的微调代码

In [ ]:
import torch
from torch import _utils
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)
from peft import PeftModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'meta-llama/Meta-Llama-3-8B'
lora_path = 'lora/llama3-8B-iepile-data2text-continue'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
model.eval()

### 现在使用的微调代码

In [2]:
import torch
from torch import _utils
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)
from peft import PeftModel
from peft import get_peft_model, LoraConfig

# 加载Llama模型和Tokenizer
model_path = "../model/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6"
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# 配置LoRA微调
lora_config = LoraConfig(
    r=8,  # LoRA参数，可以根据需要调整
    lora_alpha=16,  # LoRA参数，可以根据需要调整
    lora_dropout=0.05,  # LoRA参数，可以根据需要调整
    bias="none",  # LoRA微调的偏置选项
    task_type="CAUSAL_LM"  # 任务类型，通常使用CAUSAL_LM
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


In [3]:
from datasets import Dataset
import json

with open('../ft_data/train_data_v1.json', 'r', encoding='utf-8') as f:
    train_data_raw = json.load(f)

with open('../ft_data/val_data_v1.json', 'r', encoding='utf-8') as f:
    val_data_raw = json.load(f)


# 数据预处理：将 Instruction 和 Input 合并为输入，Response 作为输出
def preprocess_data(data, tokenizer, max_length=512):
    instructions = [item["Instruction"] for item in data]
    inputs = [item["Input"] for item in data]
    responses = [item["Response"] for item in data]

    # 合并 Instruction 和 Input
    inputs_combined = [f"{instruction}\n{input_text}" for instruction, input_text in zip(instructions, inputs)]

    # 使用统一的max_length填充，并确保inputs和labels长度一致
    model_inputs = tokenizer(inputs_combined, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
    labels = tokenizer(responses, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")["input_ids"]

    # 确保 input_ids 和 labels 长度一致
    assert model_inputs["input_ids"].shape[1] == labels.shape[1], f"Length mismatch: {model_inputs['input_ids'].shape[1]} != {labels.shape[1]}"

    return {"input_ids": model_inputs["input_ids"], "labels": labels}


# 将数据转换为合适的格式
train_data_0 = [{"Instruction": item["instruction"], "Response": item["output"], "Input": item["rule"]} for item in train_data_raw]
val_data_0 = [{"Instruction": item["instruction"], "Response": item["output"], "Input": item["rule"]} for item in val_data_raw]

# 将数据转换为合适的格式
train_data = preprocess_data(train_data_0, tokenizer)
val_data = preprocess_data(val_data_0, tokenizer)

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)


In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="../lora",  # 输出目录
    num_train_epochs=10,  # 训练轮数
    per_device_train_batch_size=2,  # 每个设备的训练批量大小
    per_device_eval_batch_size=2,  # 每个设备的评估批量大小
    evaluation_strategy="epoch",  # 每个epoch进行评估
    save_strategy="epoch",  # 每个epoch保存模型
    logging_dir="../logs",  # 日志目录
    logging_steps=10,  # 每10步记录一次日志
    save_steps=500,  # 每500步保存一次模型
    eval_steps=500,  # 每500步评估一次模型
    load_best_model_at_end=True,  # 在训练结束时加载最佳模型
    # metric_for_best_model="accuracy",  # 根据准确率选择最佳模型
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_8053/3481787107.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,0.613200,0.607305
2,0.684600,0.593674
3,0.742600,0.567335
4,0.842900,0.576224
5,0.562700,0.567671
6,0.916300,0.590891
7,0.563700,0.596283
8,0.521800,0.629527
9,0.661500,0.619426
10,0.312100,0.640178


TrainOutput(global_step=11630, training_loss=0.5403298856672909, metrics={'train_runtime': 8132.4351, 'train_samples_per_second': 2.859, 'train_steps_per_second': 1.43, 'total_flos': 5.36275143622656e+17, 'train_loss': 0.5403298856672909, 'epoch': 10.0})

In [11]:
# 进行推理
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer("在某次海上搜救任务中，搜救中心收到了一条紧急信息，一艘载有200人的客轮在远洋航行中遇到了暴风雨，导致船体严重受损，水进入船舱，船上的人员处于极度危险之中。客轮目前已失去动力，随时可能沉没。考虑到此次海上突发事件对人命安全的严重威胁，搜救中心需要判断并上报此事件为最高级别。此次事件应归类为哪个险情等级？\n海上突发事件险情分级：根据国家突发事件险情上报的有关规定，并结合海上突发事件的特点及突发事件对人命安全、海洋环境的危害程度和事态发展趋势，将海上突发事件险情信息分为特大、重大、较大、一般四级。", return_tensors="pt").to(device)
# inputs = tokenizer("Translate the following sentence to French.\nGood evening!", return_tensors="pt").to(device)
outputs = model.generate(inputs["input_ids"], max_length=512)
print('Next is the answer:\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Next is the answer:
 在某次海上搜救任务中，搜救中心收到了一条紧急信息，一艘载有200人的客轮在远洋航行中遇到了暴风雨，导致船体严重受损，水进入船舱，船上的人员处于极度危险之中。客轮目前已失去动力，随时可能沉没。考虑到此次海上突发事件对人命安全的严重威胁，搜救中心需要判断并上报此事件为最高级别。此次事件应归类为哪个险情等级？
海上突发事件险情分级：根据国家突发事件险情上报的有关规定，并结合海上突发事件的特点及突发事件对人命安全、海洋环境的危害程度和事态发展趋势，将海上突发事件险情信息分为特大、重大、较大、一般四级。
